In [10]:
import numpy as np

def scr(scr_values, correlation_matrix):
    """
    Calcola il Solvency Capital Requirement (SCR) aggregato.
    
    :param scr_values: Array numpy dei singoli SCR.
    :param correlation_matrix: Matrice di correlazione tra i rischi.
    :return: Valore aggregato dello SCR.
    """
    scr_values = np.array(scr_values)
    correlation_matrix = np.array(correlation_matrix)
    
    # Verifica che le dimensioni siano compatibili
    if scr_values.shape[0] != correlation_matrix.shape[0] or correlation_matrix.shape[0] != correlation_matrix.shape[1]:
        raise ValueError("Le dimensioni della matrice di correlazione e del vettore SCR non corrispondono.")
    
    scr_agg = np.sqrt(scr_values.T @ correlation_matrix @ scr_values)
    return scr_agg



# Shock regolatori
shock_azionario = 0.39
shock_tasso = 0.015
shock_immobiliare = 0.25
shock_valutario = 0.25
shock_spread = 0.084
soglia_concentrazione = 50
tasso_concentrazione = 0.10
shock_vita = 0.15  # 15% di shock su esposizioni Vita
shock_non_vita = 0.20  # 20% su esposizioni Non-Life
shock_salute = 0.18  # 18% su esposizioni Health

# Matrice di correlazione per il rischio di mercato (Solvency II)
corr_matrix_mercato = np.array([
    [1.00, 0.50, 0.25, 0.25, 0.50, 0.50],
    [0.50, 1.00, 0.25, 0.25, 0.50, 0.50],
    [0.25, 0.25, 1.00, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 1.00, 0.25, 0.25],
    [0.50, 0.50, 0.25, 0.25, 1.00, 0.50],
    [0.50, 0.50, 0.25, 0.25, 0.50, 1.00]
])

# Matrice di correlazione per l'aggregazione finale tra moduli di rischio
corr_matrix_finale = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.25],
    [0.25, 0.25, 1.00]
])

# Matrice di correlazione per rischi assicurativi (supponiamo correlazione moderata)
corr_matrix_assicurativi = np.array([
    [1.00, 0.30, 0.25],
    [0.30, 1.00, 0.35],
    [0.25, 0.35, 1.00]
])


In [11]:
# Definizione delle esposizioni (in milioni di euro)
esposizione_azionario = 100
esposizione_obbligazionaria = 200
esposizione_immobiliare = 50
esposizione_valutaria = 30
esposizione_spread = 100
esposizione_concentrazione = 80
esposizione_controparti = [50, 30, 40]  # Esposizioni a diverse controparti
prob_default = [0.02, 0.03, 0.015]  # Probabilità di default delle controparti
loss_given_default = [0.6, 0.5, 0.7]  # LGD per ciascuna controparte

# Esposizioni per rischi assicurativi
esposizione_vita = 500  # Life Underwriting Risk
esposizione_non_vita = 300  # Non-Life Underwriting Risk
esposizione_salute = 200  # Health Underwriting Risk

# Calcolo singoli SCR per il rischio di mercato
scr_tasso = esposizione_obbligazionaria * shock_tasso * 8
scr_azionario = esposizione_azionario * shock_azionario
scr_immobiliare = esposizione_immobiliare * shock_immobiliare
scr_valutario = esposizione_valutaria * (1 - 1 / (1 + shock_valutario))
scr_spread = esposizione_spread * shock_spread
scr_concentrazione = max(0, (esposizione_concentrazione - soglia_concentrazione) * tasso_concentrazione)

# Calcolo SCR aggregato per il rischio di mercato
scr_mercato = scr(
    [scr_tasso, scr_azionario, scr_immobiliare, scr_valutario, scr_spread, scr_concentrazione], 
    corr_matrix_mercato
)

# Calcolo SCR per rischio di controparte
scr_controparti_finale = sum(e * p * l for e, p, l in zip(esposizione_controparti, prob_default, loss_given_default))

# Calcolo SCR per rischi assicurativi
scr_vita = esposizione_vita * shock_vita
scr_non_vita = esposizione_non_vita * shock_non_vita
scr_salute = esposizione_salute * shock_salute
scr_underwriting = calcola_scr([scr_vita, scr_non_vita, scr_salute], corr_matrix_assicurativi)

# Aggregazione finale dello SCR
scr_totale = scr([scr_mercato, scr_controparti_finale, scr_underwriting], corr_matrix_finale)

# Stampa risultati
print(f"\nSCR Totale Rischio di Mercato: {scr_mercato:.2f}M")
print(f"SCR Totale Rischio di Controparte: {scr_controparti_finale:.2f}M")
print(f"SCR Totale Rischi Assicurativi: {scr_underwriting:.2f}M")
print(f"SCR Totale Aggregato: {scr_totale:.2f}M")


SCR Totale Rischio di Mercato: 69.09M
SCR Totale Rischio di Controparte: 1.47M
SCR Totale Rischi Assicurativi: 126.82M
SCR Totale Aggregato: 159.32M
